De forma sencilla se puede ver que la regla compuesta de Simpson compuesta $S_c(f)$ se escribe como:

$$S_c(f) = \displaystyle \frac{h}{3n} \left [ f(x_0) + f(x_n) + 2 \sum_{i=1}^{\frac{n}{2}-1}f(x_{2i}) + 4 \sum_{i=1}^{\frac{n}{2}}f(x_{2i-1})\right ] $$
con $h=b-a$ y $n$ número de subintervalos (par).

Nota: Los nodos para el caso de Simpson se obtienen con la fórmula: $x_i = a +\frac{i}{2}\hat{h}, \forall i=0,\dots,2n, \hat{h}=\frac{h}{n}$.

Lo siguiente sirve de ayuda. Consideramos un caso de $n=6$ subintervalos. Estos índices se multiplican por 4 y son los índices que el subproceso 0 trabajará:

In [1]:
begin=1
end=6

In [2]:
for i in range(begin,end+1,2):
    print(i)

1
3
5


Estos son los siguientes índices que se multiplican por 2 y son los índices que el subproceso 0 trabajará:

In [3]:
for i in range(begin+1,end+1,2):
    print(i)

2
4
6


In [4]:
begin=7
end=12

Los siguientes índices que se multiplican por 4 y son los índices que el subproceso 1 trabajará:

In [6]:
for i in range(begin,end+1,2):
    print(i)

7
9
11


Estos son los siguientes índices que se multiplican por 2 y son los índices que el subproceso 1 trabajará:

In [7]:
for i in range(begin+1,end+1,2):
    print(i)

8
10
12


In [8]:
import math
from multiprocessing import Pool 
from scipy.integrate import quad
import time
import multiprocessing as mp

**¿Qué restricciones se deben colocar a las variables $n, ns\text{_}p$ y $p$ para que el programa siguiente obtenga resultados correctos?**

por ejemplo si pruebas con $n=6, p=2, ns\text{_}p=6$ obtienes un error relativo de $10^{-7}$.

Para responder a la pregunta anterior, primero veamos qué pasa cuando $n=5$, $p=2$ y entonces $ns\text{_}p=5$.

Sean las siguientes variables el inicio y fin de los subprocesos 0 y 1, respectivamente:

$begin = (mi\text{_}id*ns\text{_}p)+1 = (0*5)+1=1$

$end = begin-1+ns\text{_}p=1-1+5=5$

$begin = (mi\text{_}id*ns\text{_}p)+1 = (1*5)+1=6$

$end = begin-1+ns\text{_}p=6-1+5=10$

Teniendo así los siguientes índices:

In [24]:
begin=1
end=5

índices que se multiplicarán por 4 en el subproceso 0:

In [25]:
for i in range(begin,end+1,2):
    print(i)

1
3
5


índices que se multiplicarán por 2 en el subproceso 0:

In [26]:
for i in range(begin+1,end+1,2):
    print(i)

2
4


In [27]:
begin=6
end=10

índices que se multiplicarán por 4 en el subproceso 1:

In [28]:
for i in range(begin,end+1,2):
    print(i)

6
8
10


índices que se multiplicarán por 2 en el subproceso 1:

In [29]:
for i in range(begin+1,end+1,2):
    print(i)

7
9


Los índices impares son los que se deben multiplicar por 4 y los pares por 2 y cómo se puede ver en los for's anteriores en el subproceso 0, sí se está haciendo correctamente pero, en el subproceso 1 los pares se están multiplicando 4 y los impares por 2, lo cuál es incorrecto.

Lo que se debe cumplir para que los índices impares se multipliquen por 4 y los pares por 2 es que la variable begin sea siempre impar. Es decir, esta variable tiene que ser de la forma $2k+1$:

$ begin = (mi\text{_}id*ns\text{_}p)+1 = 2*k+1$, entonces $mi\text{_}id*ns\text{_}p=2*k$

Lo que quiere decir que el producto anterior debe ser par. Para que un producto entre dos números sea par se deben cumplir los siguientes casos:

1.-$mi\text{_}id$ sea par y $ns\text{_}p$ sea par

2.-$mi\text{_}id$ sea impar y $ns\text{_}p$ sea par

3.-$mi\text{_}id$ sea par y $ns\text{_}p$ sea impar

Sin embargo, el caso 3 no se podría cumplir dado que $mi\text{_}id$ puede ser par o impar, por lo que la única opción de que ésto se cumpla es en los casos 1 y 2, es decir, que $ns\text{_}p$ sea par.

$$ns\text{_}p=\frac{2*n}{p}$$

Para que el cociente anterior sea par, dado que el numerador es par, $p$ puede ser par o impar y de igual forma $n$ puede ser par o impar, más específicamente:

$\frac{2*n}{p}=2*k$ entonces $\frac{n}{p}=k$ por lo que $n=p*k$

Otro punto importante de observar es que al algoritmo se le está aplicando la función entero, por lo que si el número de nodos $(2*n)$ no es divisible entre el número de subprocesos ($p$), se estarán perdiendo nodos por evalúar.

**Dado lo anterior, $ns\text{_}p$ debe ser par y entero y $n$ tiene que ser múltiplo de $p$ para que el programa siguiente obtenga resultados correctos.**


In [9]:
a=0
b=1
h=b-a
f=lambda x: math.exp(-x**2)

In [10]:
p=2

In [11]:
n=6
h_hat=h/n

In [12]:
nt=2*n #variable de apoyo para definir ns_p
ns_p=nt/p

In [13]:
nt

12

In [14]:
ns_p

6.0

In [16]:
ns_p=int(ns_p)
ns_p

6

In [17]:
def Scf_parallel(mi_id):
    """
    Compute numerical approximation using Simpson rule in an interval
    Nodes are generated via formula: x_i = a+i/2*h_hat for i=0,1,...,2n and h_hat=(b-a)/n
    n must be an even number
    Args:
        mi_id (in): id of subprocess: 0,1,2,...
    """
    begin = (mi_id*ns_p)+1
    end = begin-1+ns_p
    sum1=0
    #next for loop considers sum that is multiplied by 4 in the expression
    for i in range(begin,end+1,2):
        x = a+i/2*h_hat
        sum1+=f(x)
    sum1=4*sum1
    sum2=0
    #next for loop considers sum that is multiplied by 2 in the expression
    for i in range(begin+1,end+1,2):
        x = a+i/2*h_hat
        sum2+=f(x)
    sum2=2*sum2
    return sum1+sum2

if __name__=='__main__':
    start_time=time.time()
    with Pool(processes=p) as pool:
        results = pool.map(Scf_parallel,range(p))
        aprox=h/(6*n)*(f(a)+sum(results)-f(b)) 
    end_time=time.time()

**En el bloque del with ¿por qué tenemos que dividir por $\frac{}{6n}$ en lugar de por $\frac{}{3n}$ y por qué tenemos que restar $f(b)$?**

In [18]:
aprox

0.7468245263791945

In [19]:
obj, err = quad(f, a, b)

In [20]:
obj

0.7468241328124271

In [21]:
def err_relativo(aprox, obj):
    return math.fabs(aprox-obj)/math.fabs(obj) #obsérvese el uso de la librería math

In [22]:
err_relativo(aprox,obj)

5.26987211722925e-07

**¿Qué pasa si $n$ no es par? ¿qué tendríamos que modificar en la implementación si no es par?**